In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gp
import matplotlib as mpl
from matplotlib import ticker
import glob
from tqdm import tqdm
mpl.style.use('bmh')

## Data

see data availability

In [ ]:
polygons = # array of strings, names of departments in the country

dept = # Dataframe containing the population of departments

dept_areas = # Dataframe containing areas of departments

path_array = # array of file paths in the folder containing colocation data

N = len(polygons)

## Compute operators

build a dictionary to match patches names in the different data

In [ ]:
dict ={}
for p, pol in enumerate (polygons):
    rep=pol.replace('Ã©', 'e')
    rep_=rep.replace('Ã¨','e')
    rep__=rep_.replace('Ã´','o')
    rep___=rep__.replace('ô','o')
    rep____=rep___.replace('è','e')
    rep_____=rep____.replace('é','e')
    dict[pol]=rep_____.upper()

compute reproduction operators

In [ ]:
def Rij_computer (path):
    
    data = pd.read_csv(path, delimiter=',',engine='python')
    FR_data=data[data['country']==country]
    
    Rij=np.zeros((len(polygons),len(polygons)))
    for l in range (len(FR_data)):
        i=FR_data.iloc[l].loc['polygon1_name']
        I=np.where(polygons==i)
        j=FR_data.iloc[l].loc['polygon2_name']
        J=np.where(polygons==j)
        pop=dept[dept['dept name']==dict[i]].loc[:,'pop 2020'].iloc[0]
        Rij[I[0][0]][J[0][0]]=FR_data.iloc[l].loc['link_value']*pop
    
    return Rij

def Rij_plotter (Rij):
    
    fig,ax=plt.subplots(figsize=(18,18))
    im = ax.imshow(Rij,norm=mpl.colors.LogNorm())
    cbar = fig.colorbar(im,shrink=.7)
    cbar.ax.set_ylabel(r'$R$',rotation=90,fontsize=20)
    ax.set_xticks(np.arange(len(polygons)))
    ax.set_yticks(np.arange(len(polygons)))
    ax.set_xticklabels(polygons,rotation=90)
    _=ax.set_yticklabels(polygons)
    plt.tight_layout()
    plt.show()

In [ ]:
Rij_vec=[]
for path_ in tqdm(path_arr):
    Rij_vec.append(Rij_computer(path_))

In [ ]:
cal=[]
for path_ in (path_arr):
    cal.append(path_[-14:-4])
calendar=np.array(cal)
print(len(calendar))

In [ ]:
Rij_plotter(Rij_vec[20])

## StayPut Correction

read stayput data from 2020 and 2021, aggregate them weekly with the same convention as colocation data, concatenate

In [ ]:
path_stayput_2020= # path
path_stayput_2021= # path
data_stayput_2020 = pd.read_csv(path_stayput_2020,engine='python',sep="\t")
data_stayput_2021 = pd.read_csv(path_stayput_2021,engine='python',sep="\t")

stayput_2020=data_stayput_2020[data_stayput_2020['country']==country_sp]
stayput_2021=data_stayput_2021[data_stayput_2021['country']==country_sp]

stayput_2020['ds'] = pd.to_datetime(stayput_2020['ds'])
stayput_2020 = stayput_2020.groupby(['polygon_name', pd.Grouper(key='ds', freq='W-MON')])['all_day_ratio_single_tile_users'].sum().reset_index().sort_values('ds')
stayput_2020['ds'] = pd.to_datetime(stayput_2020['ds']) - pd.to_timedelta(6, unit='d')

stayput_2021['ds'] = pd.to_datetime(stayput_2021['ds'])
stayput_2021 = stayput_2021.groupby(['polygon_name', pd.Grouper(key='ds', freq='W-MON')])['all_day_ratio_single_tile_users'].sum().reset_index().sort_values('ds')
stayput_2021['ds'] = pd.to_datetime(stayput_2021['ds']) - pd.to_timedelta(6, unit='d')

stayput = pd.concat([stayput_2020, stayput_2021])

build a new dictionary to match patches names in different data

In [ ]:
dict_pol_reg={}
dict_pol_reg_={}
for p, pol in enumerate (polygons):
    reg_name=dept[dept['dept name']==dict[pol]].loc[:,'reg name'].iloc[0]

    r=reg_name.replace('Ã´','-')
    r_=r.replace('Ã©','-')
    r__=r_.replace('ÃŽ','-')
    r___=r__.replace('Ã¨','-')
    r____=r___.replace('Î','-')
    r_____=r____.replace('é','-') 
    r______=r_____.replace('ô','-')
    dict_pol_reg_[pol]=r______

    r=reg_name.replace('Î','ÃŽ')
    r_=r.replace('é','Ã©') 
    r__=r_.replace('ô','Ã´')
    dict_pol_reg[pol]=r__

correct reproduction operators

In [ ]:
Rij_vec_sp=Rij_vec.copy()

for r,R in tqdm(enumerate(Rij_vec)):
    for l in range (len(R)):
        pol=polygons.iloc[l]
        reg=dict_pol_reg[pol]
        reg_=dict_pol_reg_[pol]
        #print(reg,reg_,pd.to_datetime(path_arr[r][-14:-4]))
        pop=dept[dept['dept name']==dict[pol]].loc[:,'pop 2020'].iloc[0]
        dep_area=float(dept_areas[dept_areas['DEPARTMENT']==dict[pol]].loc[:,'AREA'].iloc[0])
        d=pop/dep_area
        
        if (pd.to_datetime(path_arr[r][-14:-4])>pd.to_datetime('2020-08-25')):
            a=stayput[stayput['polygon_name']==reg]
        else:
            a=stayput[stayput['polygon_name']==reg_]
        
        sp7=a[a['ds']==path_arr[r][-14:-4]].loc[:,'all_day_ratio_single_tile_users'].iloc[0]
        sp=sp7/7
        R_sp=sp*(d*0.36)
        Rij_vec_sp[r][l][l]-=R_sp

In [ ]:
Rij_plotter(Rij_vec_sp[20])

## Compare

In [ ]:
polygons_ind=[]
for p, pol in enumerate (polygons):
    polygons_ind.append(dept[dept['dept name']==dict[pol]].loc[:,'dept'].iloc[0])
polygons_ind=np.array(polygons_ind)

In [ ]:
a='2021-02-02'
Rij_uncorrected=Rij_vec[np.where(calendar==a)[0][0]]
Rij_corrected=Rij_vec_sp[np.where(calendar==a)[0][0]]
correction=[Rij_uncorrected[i][i]-Rij_corrected[i][i] for i in range(len(polygons))]
relative_correction=[(Rij_uncorrected[i][i]-Rij_corrected[i][i])/Rij_uncorrected[i][i] for i in range(len(polygons))]

In [ ]:
shp_filename = # name of the shp file 
shp = gp.read_file(shp_filename)
ss = '+proj=lcc +lat_1=49 +lat_2=44 +lat_0=46.5 +lon_0=3 +x_0=700000 +y_0=6600000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs '
shp = shp.to_crs(ss)

# Remove Corsica
shp.drop(shp.index[shp['dept'] == '2A'], inplace = True)
shp.drop(shp.index[shp['dept'] == '2B'], inplace = True)
shp=shp.reset_index(drop=True)

shp = shp.assign(centroid=shp.centroid)

shp['dept'] = shp['dept'].astype('category')
shp['dept'] = shp['dept'].cat.set_categories(polygons_ind)
shp=shp.sort_values('dept').reset_index(drop=True)

In [ ]:
shp_correction=shp.copy()
shp_correction['correction']=correction
shp_correction['relative_correction']=relative_correction
shp_correction.head()

In [ ]:
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = .5
plt.rcParams['ytick.minor.width'] = .5

min_value_=shp_correction.loc[:,'correction'].min()
max_value_=shp_correction.loc[:,'correction'].max()
fig,ax=plt.subplots(figsize=(5,5),dpi=150)
cmap='Purples'
shp_correction.plot(ax=ax, column='correction', lw=0.2, legend=False,cmap=cmap,norm=mpl.colors.LogNorm(vmin=min_value_,vmax=max_value_))
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.LogNorm(vmin=min_value_,vmax=max_value_))
sm.set_array([])
cbar = plt.colorbar(sm,shrink=.7,ax=ax) 
minor_locator = ticker.LogLocator(numticks=10)
cbar.ax.yaxis.set_major_locator(minor_locator)
ax.axis('off')
ax.set_title('stayput correction',fontsize=12)

In [ ]:
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = .5
plt.rcParams['ytick.minor.width'] = .5

min_value_=shp_correction.loc[:,'relative_correction'].min()
max_value_=shp_correction.loc[:,'relative_correction'].max()
fig,ax=plt.subplots(figsize=(5,5),dpi=150)
cmap='Purples'
shp_correction.plot(ax=ax, column='relative_correction', lw=0.2, legend=False,cmap=cmap,norm=mpl.colors.LogNorm(vmin=min_value_,vmax=max_value_))
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.LogNorm(vmin=min_value_,vmax=max_value_))
sm.set_array([])
cbar = plt.colorbar(sm,shrink=.7,ax=ax) 
minor_locator = ticker.LogLocator(numticks=10)
cbar.ax.yaxis.set_major_locator(minor_locator)
ax.axis('off')
ax.set_title('stayput relative correction',fontsize=12)